In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import pandas as pd
from collections import Counter
import pafy
from user_agents import parse
import json
%pylab inline
import time
import os

Populating the interactive namespace from numpy and matplotlib


In [2]:
MOOCCompleters = pd.read_csv("completed_students.csv", header = None)
MOOCCompleters = MOOCCompleters.iloc[:,0].tolist()
print(len(MOOCCompleters),"completers")

video_ids_df = pd.read_csv("id-codes.csv")
id_code_dict = {video_ids_df.id.iloc[i]:video_ids_df.code.iloc[i] for i in range(video_ids_df.shape[0])}

df_video_descr = pd.read_csv("Stats_Videos.csv")
videoname_dict = {}
videoduration_dict = {}
for v in range(df_video_descr.shape[0]):
    _code = df_video_descr.URL.iloc[v].split("/")[-1]
    videoname_dict[_code] = df_video_descr.Title.iloc[v]
    _min= (df_video_descr.Minutes.iloc[v])
    _sec = (df_video_descr.Seconds.iloc[v])
    if np.isnan(_min):
        _min = 0
    if np.isnan(_sec):
        _sec = 0
    try:
        videoduration_dict[_code] = _min*60 + _sec
    except:
        print(df_video_descr.iloc[v])
        
video_events = ['load_video', 'pause_video', 'play_video', 'seek_video', 'speed_change_video', 'stop_video']

1879 completers


In [3]:
basedir = "clickdata/"
clickfiles = [f for f in os.listdir(basedir) if ('csv' in f)]
print(len(clickfiles), "\tclickstream log files found")
clickfiles = sorted(clickfiles)

64 	clickstream log files found


In [4]:
def GetVideoDuration(_video_id,_youtube_id,_id_code_dict,_video_duration_dict):
    """if video duration isn't recorded, look it up on youtube and update dict once"""
    try:
        if _youtube_id in ["html5","mobile"]: #in these cases, the youtube ID is harder to get
            if _video_id in _id_code_dict:
                _youtube_id_real = _id_code_dict[_video_id]
                _duration = _video_duration_dict[_youtube_id_real]
            else:
                _duration = -1
        elif _youtube_id in _video_duration_dict:
            _duration = _video_duration_dict[_youtube_id]
        else:
            # this youtube_id is perhaps a youtube ID John missed
            # let's go over to Youtube and get the duration for it
            try:
                _vid = GetYoutubeVidDuration(_youtube_id)
                _duration = _vid["duration"]
                _name = _vid["name"]
                print("Unseen video added:", _youtube_id, ", titled: ", _name, " (",str(_duration),"s)")
                # add it to the KNOWN durations so that you don't run this multiple times
                _video_duration_dict[_youtube_id] = _duration
            except:
                _duration = -1
                print("Can't get the duration for code: ", _code)
                pass
    except:
        _duration = -1
        print("Unforeseen error. This should never happen. Breaking control flow here intentionally.")
    return _duration, _video_duration_dict

def GetEngagementFeatures(_df_vid,_id_code_dict,_video_duration_dict,verbose=False):
    """
    _df_vid = raw log of all video events
    _id_code_dict = maps internal edX ID codes to YouTube IDs
    _video_duration_dict = maps YouTube ID codes to duration in seconds
    """
    
    _starttime = time.time()
    _df_eng = []
    _df_vid_details = pd.DataFrame(_df_vid.student_id)
    _df_vid_details["video_id"] = [json.loads(i)["id"] for i in _df_vid.event]
    _df_vid_details["youtube_id"] = [json.loads(i)["code"] for i in _df_vid.event]
    _df_vid_details["duration"] = [0]*_df_vid.shape[0]
    print(_df_vid_details.shape)
    _df_vid_details.drop_duplicates(inplace=True)
    print(_df_vid_details.shape)
    
    for i in range(_df_vid_details.shape[0]):
        if verbose:
            if (i%1000==0):
                print("\t\t",i," out of ",_df_vid_details.shape[0],"video interactions processed")
        _student = _df_vid_details.student_id.iloc[i]
        _video_id = _df_vid_details.video_id.iloc[i]
        _youtube_id = _df_vid_details.youtube_id.iloc[i]
        _duration,_video_duration_dict = GetVideoDuration(_video_id,_youtube_id,_id_code_dict,_video_duration_dict)
        _df_vid_details["duration"].iloc[i] = _duration
        #once duration has been extracted, get engagement metrics for that student-video pair
        _df_eng.append(EngagementMetricsStudent(_studentid=_student, _videoid=_video_id,_video_duration=int(_duration), _df=_df_vid))
    #_df_vid_details.drop_duplicates(inplace=True)
    _df_eng = pd.DataFrame(_df_eng)
    if verbose:
        print("\t\t(time:",str(time.time() - _starttime),")")
    return _df_eng, _video_duration_dict

In [6]:
metadata_dict = {"studentnums":[],"dates":[],"studentIDs":[],"videostudentnums":[],\
                 "videonums":[],"clicknums":[],"videoclicknums":[],"videoviewerstudentIDs":[]}
ua_history = [] # stores dicts of all unique student_id:user_agent pairs for day i in index i
ua_vid_history = []

ua_running_dict = {} # constantly updated dict of all student_id:[user_agents...]
engagement_dfs = [] #not storing it here because dataframes get BULKY. Writing to disk instead

filenum = 0
for filenum,f in enumerate(clickfiles):
    print(filenum,f)
    print("\tDate Extracted: ",parse_date_from_filename(f))
    _df = pd.read_csv(basedir+f, compression='gzip', header = 'infer')
    _df_vid = _df[_df['event_type'].isin(video_events)]
    metadata_dict = UpdateMetadata(_df,_df_vid,f,metadata_dict)
    
    # extract user agent data
    UA_df, ua_running_dict = GetUserAgentFeatures(_df,ua_running_dict,verbose=True)
    UA_df.to_csv("newsubmission/ua/"+str(parse_date_from_filename(f))+".csv",index=False)
    
    # extract engagement features
    eng_df,videoduration_dict = GetEngagementFeatures(_df_vid,id_code_dict,videoduration_dict,verbose=True)
    eng_df.to_csv("newsubmission/eng/"+str(parse_date_from_filename(f))+".csv",index=False)

0 notredamex-edx-events-2015-04-15[parsed][anonymized].csv.gz
	Date Extracted:  2015-04-15
		 0 out of  2574  students processed
		 1000 out of  2574  students processed
		 2000 out of  2574  students processed
		(time: 2.9643213748931885 )
(47306, 4)
(8938, 4)
		 0  out of  8938 video interactions processed


/afs/crc.nd.edu/user/s/snagrech/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Unseen video added: SbJkyr59OYw , titled:  NDXSOC12T115-V010800  ( 468 s)
Unseen video added: guCMaiH433Q , titled:  NDXSOC12T115-V018000  ( 255 s)
		 1000  out of  8938 video interactions processed
		 2000  out of  8938 video interactions processed
		 3000  out of  8938 video interactions processed
		 4000  out of  8938 video interactions processed
		 5000  out of  8938 video interactions processed
Unseen video added: 3_yD_cEKoCk , titled:  edX – How It Works  ( 141 s)
		 6000  out of  8938 video interactions processed
Unseen video added: hZWuH9XOq3o , titled:  NDXSOC12T115-V009100  ( 305 s)
		 7000  out of  8938 video interactions processed
		 8000  out of  8938 video interactions processed
		(time: 238.35365462303162 )
1 notredamex-edx-events-2015-04-16[parsed][anonymized].csv.gz
	Date Extracted:  2015-04-16
		 0 out of  4344  students processed
		 1000 out of  4344  students processed
		 2000 out of  4344  students processed
		 3000 out of  4344  students processed
		 4000 out of  

In [5]:
def parse_date_from_filename(fname):
    try:
        fname = fname.replace('[parsed][anonymized].csv.gz','')
        fname = fname.split('-')
        datestring = '-'.join([fname[3] , fname[4] , fname[5]])
        return datestring
    except:
        return "I tried :("
    
def UpdateMetadata(_df,_df_vid,_filename,metadata,verbose=False):
    """updates various attributes for metadata by appending newer attributes at the end of each list"""
    _starttime = time.time()
    try:
        metadata["clicknums"].append(_df.shape[0])
        metadata["dates"].append(parse_date_from_filename(_filename))
        metadata["studentnums"].append(len(np.unique(_df.student_id)))
        metadata["studentIDs"].append(np.unique(_df.student_id))
        
        #now for video events
        
        metadata["videoclicknums"].append(_df_vid.shape[0])
        metadata["videonums"].append(len(np.unique(_df_vid.page)))
        metadata["videostudentnums"].append(len(np.unique(_df_vid.student_id)))
        metadata["videoviewerstudentIDs"].append(np.unique(_df_vid.student_id))
        
    except:
        print("Error updating metadata for file:\t", _filename)
        if verbose:
            print("\t\t(time:",str(time.time() - _starttime),")")
        return metadata
    if verbose:
        print("\t\t(time:",str(time.time() - _starttime),")")
    return metadata

def GetUserAgentFeatures(_df,_ua_running_dict,verbose=False):
    """create user-agent featuresets for each student for each day"""
    _starttime = time.time()
    _updated_ua_running_dict = _ua_running_dict
    _df_pairs = _df[["student_id","agent"]].drop_duplicates()
    for _i in range(_df_pairs.shape[0]):
        _student = _df_pairs.student_id.iloc[_i]
        _agent = _df_pairs.agent.iloc[_i]
        if _df_pairs.student_id.iloc[_i] in _ua_running_dict:
            if _agent not in _updated_ua_running_dict[_student]:
                _updated_ua_running_dict[_student].append(_agent)
            else:
                pass
        else:
            _updated_ua_running_dict[_student] = [_agent]
    _df_ua = []
    for _s,_student in enumerate(_updated_ua_running_dict.keys()):
        if verbose:
            if _s%1000==0:
                print("\t\t",str(_s),"out of ",str(len(_updated_ua_running_dict))," students processed")
        _ua_parsed = ParseUserAgentAttrs(_updated_ua_running_dict[_student])
        _ua_parsed["ID"] = _student
        _df_ua.append(_ua_parsed)
    _df_ua = pd.DataFrame(_df_ua)
    _df_ua.fillna(0,inplace=True)
    if verbose:
        print("\t\t(time:",str(time.time() - _starttime),")")
    return _df_ua,_updated_ua_running_dict

def ParseUserAgentAttrs(list_of_uas):
    """given a list of user agents for a student, returns a dict of various attributes"""
    _feature_dict = {}
    
    _feature_dict['num_devices'] = len(list_of_uas)
    _num_phone = 0
    _num_tablet = 0
    _num_pc = 0
    _num_touch = 0
    
    for ua in list_of_uas:
        if isinstance(ua,float):
            #then it's a nan...
            continue
        else:
            _parsed_ua = parse(ua)
            _parsed_browser = _parsed_ua.browser.family
            _parsed_os = _parsed_ua.os.family
            _parsed_device = _parsed_ua.device.family
            if " " in _parsed_browser:
                _parsed_browser = _parsed_browser.split(" ")[0]
            if " " in _parsed_os:
                _parsed_os = _parsed_os.split(" ")[0]
            if " " in _parsed_device:
                _parsed_device = _parsed_device.split(" ")[0]
            if "," in _parsed_browser:
                _parsed_browser = _parsed_browser.split(",")[0][:-1]
            if "," in _parsed_os:
                _parsed_os = _parsed_os.split(",")[0][:-1]
            if "," in _parsed_device:
                _parsed_device = _parsed_device.split(",")[0][:-1]
            if ("browser_" + str(_parsed_browser)) not in _feature_dict:
                _feature_dict["browser_" + str(_parsed_browser)] = 1
            if ("os_" + str(_parsed_os)) not in _feature_dict:
                _feature_dict["os_" + str(_parsed_os)] = 1
            if ("device_" + str(_parsed_device)) not in _feature_dict:
                _feature_dict["device_" + str(_parsed_device)] = 1
            if _parsed_ua.is_mobile:
                _num_phone+=1
            if _parsed_ua.is_tablet:
                _num_tablet+=1
            if _parsed_ua.is_pc:
                _num_pc+=1
            if _parsed_ua.is_touch_capable:
                _num_touch+=1
    _feature_dict['num_phone'] = _num_phone
    _feature_dict['num_tablet'] = _num_tablet
    _feature_dict['num_pc'] = _num_pc
    _feature_dict['num_touch'] = _num_touch
    
    return _feature_dict


def get_number_plays(student_id,video_id,_df):
    _df=_df.loc[_df["student_id"]==student_id]
    _df.head()
    count=0
    _df_playvids = _df.loc[_df.event_type=="play_video",:]
    numrows = _df_playvids.shape[0]
    for i in range(numrows):
        event_of_first_row= _df_playvids.event.iloc[i]
        first_row_json=json.loads(event_of_first_row)
        if(first_row_json['id']==video_id):
            count=count+1
    return count

def get_forwards(student_id,video_id,_df):
    _df=_df.loc[_df["student_id"]==student_id]
    count=0
    starttime = 0
    endtime = 0
    jumptime=0
    videoid = 0
    _df_seekvids = _df.loc[_df.event_type=="seek_video",:]
    numrows = _df_seekvids.shape[0]
    for i in range(numrows):
        event_of_first_row=_df_seekvids.event.iloc[i]
        first_row_json=json.loads(event_of_first_row)
        if(first_row_json['id']==video_id):
            starttime=first_row_json['old_time']
            endtime=first_row_json['new_time']
            jumptime=first_row_json['new_time']-first_row_json['old_time']
            videoid=first_row_json['id']
            if(first_row_json['old_time']<first_row_json['new_time']):
                count=count+1
    return count

def get_rewinds(student_id,video_id,_df):
    _df=_df.loc[_df["student_id"]==student_id]
    count=0
    start_time=0
    end_time=0
    jump_time=0
    video_id=0
    _df_seekvids = _df.loc[_df.event_type=="seek_video",:]
    numrows = _df_seekvids.shape[0]
    for i in range(numrows):
        event_of_first_row=_df_seekvids.event.iloc[i]
        first_row_json=json.loads(event_of_first_row)
        if(first_row_json['id']==video_id):
            start_time=first_row_json['old_time']
            end_time=first_row_json['new_time']
            jump_time=first_row_json['new_time']-first_row_json['old_time']
            video_id=first_row_json['id']
            if(first_row_json['old_time']>first_row_json['new_time']):
                count=count+1   
    return count

def get_speed_changes(student_id,video_id,_df):
    _df=_df.loc[_df["student_id"]==student_id]
    count=0
    _df_speed_change = _df.loc[_df.event_type=="speed_change_video"]
    numrows = _df_speed_change.shape[0]
    for i in range(numrows):
        event_of_first_row = _df_speed_change.event.iloc[i]
        first_row_json=json.loads(event_of_first_row)
        if(first_row_json['id']==video_id):
            count=count+1
    return count

def get_seeks(student_id,video_id,_df):
    _df=_df.loc[_df["student_id"]==student_id]
    seek_list = []
    start_time = 0
    end_time=0
    jump_time=0
    _df_seek = _df.loc[_df.event_type=="seek_video",:]
    numrows = _df_seek.shape[0]
    for i in range(numrows):
        event_of_first_row = _df_seek.event.iloc[i]
        first_row_json = json.loads(event_of_first_row)
        if(first_row_json['id']==video_id):
            start_time = first_row_json['old_time']
            end_time = first_row_json['new_time']
            jump_time = first_row_json['new_time'] - first_row_json['old_time']
            video_id = first_row_json['id']
            _dict = {'student_id':student_id,\
                    'video_id':video_id,\
                    'start_time':start_time,\
                    'end_time':end_time,\
                    'jump':jump_time}
            seek_list.append(_dict)
    return seek_list

def get_percent_video_completed(student_id,video_id,total_time,_df):
    _df = _df.loc[_df["student_id"]==student_id]
    current_time = 0
    _df_stop = _df.loc[_df.event_type=="stop_video",:]
    numrows = _df_stop.shape[0]
    for i in range(numrows):
        event_of_first_row = _df_stop.event.iloc[i]
        first_row_json = json.loads(event_of_first_row)
        if(first_row_json['id']==video_id):
            if (first_row_json["currentTime"]>current_time):
                current_time=first_row_json['currentTime']
    return (current_time) * 100 / total_time

def video_timeline(student_id,video_id,total_time,_df):
    flag=0
    previous_event = 'stop'
    _df=_df.loc[_df["student_id"]==student_id]
    video_events = ['load_video', 'pause_video', 'play_video', 'seek_video', 'speed_change_video', 'stop_video']
    _df_video_events = _df[_df['event_type'].isin(video_events)]
    seconds=[0 for i in range(total_time)]
    numrows = _df_video_events.shape[0]
    for i in range(numrows-1, -1, -1):
        event_of_first_row=_df_video_events.event.iloc[i]
        first_row_json=json.loads(event_of_first_row)
        _event_type = _df_video_events.event_type.iloc[i]
        if(_event_type == 'play_video'):
            if(first_row_json['id'] == video_id):
                if previous_event == 'play':
                    for i in range(int(flag),int(first_row_json['currentTime'])):
                        seconds[i]=seconds[i]+1
                    flag=first_row_json['currentTime']
                else:
                    flag=first_row_json['currentTime']
                    previous_event='play'
        elif(_event_type == 'stop_video'):
            if(first_row_json['id']==video_id):
                if previous_event == 'play':
                    for i in range(int(flag),int(first_row_json['currentTime'])):
                        seconds[i] = seconds[i] + 1
                    previous_event='stop'
                    flag=first_row_json['currentTime']
                else:
                    flag=first_row_json['currentTime']
                    previous_event='stop'
        elif(_event_type == 'pause_video'):
            if(first_row_json['id']==video_id):
                if previous_event == 'play':
                    for i in range(int(flag),int(first_row_json['currentTime'])):
                        seconds[i] = seconds[i] + 1
                    previous_event='pause'
                    flag=first_row_json['currentTime']
                else:
                    flag=first_row_json['currentTime']
                    previous_event='pause'
        elif(_event_type == 'seek_video'):
            if(first_row_json['id']==video_id):
                if previous_event == 'play':
                    for i in range(int(flag),int(first_row_json['new_time'])):
                        seconds[i] = seconds[i] + 1
                flag=first_row_json['old_time']
                previous_event='seek'
    return seconds

def get_percent_video_engagement(student_id,video_id,total_time,_df):
    seconds=video_timeline(student_id,video_id,total_time,_df)
    counter=0
    for i in range(0, len(seconds)):
        counter=seconds[i] + counter
    return float(counter)*100 / len(seconds)

def get_percent_video_rewatched(student_id,video_id,total_time,df):
    seconds=video_timeline(student_id,video_id,total_time,df)
    counter=0
    for i in range(0,len(seconds)):
        if (seconds[i]>1):
            counter=counter+1
    return float(counter)*100/len(seconds)

def get_percent_video_watched(student_id,video_id,total_time,df):
    seconds=video_timeline(student_id,video_id,total_time,df)
    counter=0
    for i in range(0,len(seconds)):
        if (seconds[i]>0):
            counter=counter+1
    return float(counter)*100/len(seconds)

def get_max_times_video_watched(student_id,video_id,total_time,df):
    seconds=video_timeline(student_id,video_id,total_time,df)
    return max(seconds)

def EngagementMetricsStudent(_studentid,_videoid,_video_duration,_df):
    metricnames = ['numberplays','forwards','rewinds','speedchanges', \
                   'numberseeks','seeks_time', 'perc_watched','perc_rewatch']
    engagementmetrics = {k:0 for k in metricnames}
    engagementmetrics['student_id'] = _studentid
    engagementmetrics['video_id'] = _videoid
    try:
        engagementmetrics['numberplays'] = get_number_plays(_studentid, _videoid, _df)
    except:
        engagementmetrics['numberplays'] = -1
    try:
        engagementmetrics['forwards'] = get_forwards(_studentid, _videoid, _df)
    except:
        engagementmetrics['forwards'] = -1
    try:
        engagementmetrics['rewinds'] = get_rewinds(_studentid, _videoid, _df)
    except:
        engagementmetrics['rewinds'] = -1
    try:
        engagementmetrics['speedchanges'] = get_speed_changes(_studentid, _videoid, _df)
    except:
        engagementmetrics['speedchanges'] = -1
#     try:
#         engagementmetrics['seeks'] = get_seeks(_studentid, _videoid, _df)
#     except:
#         engagementmetrics['seeks'] = -1
    #rest of the engagement features to be added when runtimes for video ID are made available
    #engagementmetrics['perc_completed'] = get_seeks(_studentid, _videoid, _video_duration, _df)
    seeklist = []
    try:
        seeklist = get_seeks(_studentid, _videoid, _df)
    except:
        print("seeklist failed :(")
        engagementmetrics['numberseeks'] = 0
        engagementmetrics['seeks_time'] = 0.0
        pass
    engagementmetrics['numberseeks'] = len(seeklist)
    engagementmetrics['seeks_time'] = sum([i["jump"] for i in seeklist])
    try:
        engagementmetrics['perc_watched'] = get_percent_video_watched(_studentid, _videoid, int(_video_duration), _df)
    except:
        engagementmetrics['perc_watched'] = -1
    try:
        engagementmetrics['perc_rewatch'] = get_percent_video_rewatched(_studentid, _videoid, int(_video_duration),_df)
    except:
        engagementmetrics['perc_rewatch'] = -1
    return engagementmetrics

def GetYoutubeVidDuration(youtubeID):
    _vid = {"duration":-1,"name":""}
    try:
        _url = "https://www.youtube.com/watch?v=" + youtubeID
        _video = pafy.new(_url)
        _vid["duration"] = _video.length
        _vid["name"] = _video.title
    except:
        pass
    return _vid